In [ ]:
start = time.time()

Rsults = {}
# Assuming 'test' is your DataFrame
unique_users = test['sparse_user_id'].unique()
np.random.seed(42)
# Randomly select 1000 users
selected_users = np.random.choice(unique_users, size=50, replace=False)
for i, user in enumerate(selected_users):
    print(f"user n°{i+1}/{len(selected_users)}")
    new_ratings = list(
        zip(
            test[test['sparse_user_id'] == user]['sparse_movie_id'],
            test[test['sparse_user_id'] == user]['rating']
        )
    )
    df_result = test.groupby('sparse_movie_id')['sparse_user_id'].agg(list).reset_index()
    weights_map = {user: weights for weights, user in zip(weights, indices)}
    k = 30  # Number of top weights to keep
    df_result['sorted_weights_and_ids'] = df_result['sparse_user_id'].apply(
        lambda user_ids: heapq.nlargest(
            k,
            [(user_id, weights_map.get(user_id, 0)) for user_id in user_ids if weights_map.get(user_id, 0) > 0],
            key=lambda x: x[1]  # Sort by weights
        )
    )
    RRR = []
    for i, movie_to_rate in enumerate(test[test['sparse_user_id'] == user]['sparse_movie_id'].values):
        df_new_rating = pd.DataFrame(df_result.loc[movie_to_rate, 'sorted_weights_and_ids'], columns=["userId", "Weights"])
        # Filter rows where sparse_movie_id = 0
        df_filtered_ratings = df_ratings[df_ratings["sparse_movie_id"] == movie_to_rate]
        
        # Merge on sparse_user_id and ID
        df_with_ratings = df_new_rating.merge(
            df_filtered_ratings[["sparse_user_id", "rating"]],
            how="left",
            left_on="userId",
            right_on="sparse_user_id"
        )
        
        # Drop extra column if desired
        df_with_ratings = df_with_ratings.drop(columns=["sparse_user_id"])
        #print(df_with_ratings)
    
        mean_ratings = df_ratings[df_ratings['sparse_user_id'].isin(df_with_ratings['userId'])] \
            .groupby('sparse_user_id')['rating'].mean()
        
        # Map the computed mean ratings back to df_with_ratings
        df_with_ratings['mean_rating'] = df_with_ratings['userId'].map(mean_ratings)
        r_u = df_ratings[df_ratings['sparse_user_id'] == 0]['rating'].mean()
        # Compute the weighted difference
        df_with_ratings['weighted_diff'] = df_with_ratings['Weights'] * (df_with_ratings['rating'] - df_with_ratings['mean_rating'])
        
        # Compute the numerator: sum of weighted differences
        numerator = df_with_ratings['weighted_diff'].sum()
        
        # Compute the denominator: sum of absolute weights
        denominator = df_with_ratings['Weights'].abs().sum()
        if denominator == 0:
            print(f"User n°{user} skipped (denominator is zero)")
            continue
        
        # Compute the final result
        result = numerator / denominator if denominator > 0 else 0
        RRR.append((movie_to_rate, result + r_u))
        #print(f"{i+1}/{len(test[test['sparse_user_id'] == 0]['sparse_movie_id'].values)}")
        #print(movie_to_rate)
        #print(f"Result: {result + r_u}")
        #print(20*"-")
    Rsults[user] = RRR
print(time.time()-start)

In [ ]:
import numpy as np

def add_predicted_ratings(results, selected_users_df):
    # Create a copy of the DataFrame to avoid modifying the original
    df = selected_users_df.copy()
    
    # Function to get the predicted rating for a user-movie pair
    def get_predicted_rating(user, movie):
        # Check if the user exists in the results
        if user in results:
            # Find the tuple with the movie ID
            matches = [rating for (m_id, rating) in results[user] if m_id == movie]
            # Return the rating if found, otherwise None
            return matches[0] if matches else None
        return None
    
    # Add predicted rating column
    df['predicted_rating'] = df.apply(lambda row: get_predicted_rating(row['sparse_user_id'], row['sparse_movie_id']), axis=1)
    
    # Add error column (absolute difference between true and predicted rating)
    df['rating_error'] = np.abs(df['rating'] - df['predicted_rating'])
    
    return df

# Use the function
selected_users_df_with_predictions = add_predicted_ratings(Rsults, selected_users_df).dropna()
selected_users_df_with_predictions.sort_values(by='sparse_user_id')

In [ ]:
def optimize_recommendation_calculation(test, df_ratings, weights, indices, num_users=100, k=30):
    np.random.seed(42)
    unique_users = test['sparse_user_id'].unique()
    selected_users = np.random.choice(unique_users, size=num_users, replace=False)
    
    # Create weights map once
    weights_map = dict(zip(indices, weights))
    
    # Precompute global mean rating
    r_u = df_ratings['rating'].mean()
    
    # Precompute movie-user mapping
    movie_users = test.groupby('sparse_movie_id')['sparse_user_id'].agg(list)
    
    # Precompute user mean ratings
    user_mean_ratings = df_ratings.groupby('sparse_user_id')['rating'].mean()
    user_std_ratings = df_ratings.groupby('sparse_user_id')['rating'].std()
    
    results = {}
    for user in selected_users:
        user_movies = test[test['sparse_user_id'] == user]['sparse_movie_id'].values
        user_results = []
        
        for movie in user_movies:
            # Get top K weighted users for this movie
            movie_users_list = movie_users.get(movie, [])
            top_users = heapq.nlargest(
                k, 
                [(uid, weights_map.get(uid, 0)) for uid in movie_users_list if weights_map.get(uid, 0) > 0],
                key=lambda x: x[1]
            )
            
            # Create a copy of the filtered DataFrame to avoid SettingWithCopyWarning
            user_ratings = df_ratings[
                (df_ratings['sparse_movie_id'] == movie) & 
                (df_ratings['sparse_user_id'].isin([u[0] for u in top_users]))
            ].copy()
            
            if len(user_ratings) == 0:
                continue
            
            # Add columns using .loc to avoid warnings
            user_ratings.loc[:, 'user_weight'] = user_ratings['sparse_user_id'].map(dict(top_users))
            user_ratings.loc[:, 'mean_rating'] = user_ratings['sparse_user_id'].map(user_mean_ratings)
            user_ratings.loc[:, 'std_rating'] = user_ratings['sparse_user_id'].map(user_std_ratings)
            user_ratings.loc[:, 'weighted_diff'] = user_ratings['user_weight'] * (user_ratings['rating'] - user_ratings['mean_rating'])
            
            numerator = user_ratings['weighted_diff'].sum()
            denominator = user_ratings['user_weight'].abs().sum()
            
            if denominator > 0:
                result = numerator / denominator + r_u
                user_results.append((movie, result))
        
        results[user] = user_results
    
    return results